In [1]:
import sqlalchemy as db
from sqlalchemy import create_engine, inspect, MetaData
import pandas as pd

In [2]:
engine = db.create_engine('sqlite:///C:\\Users\\peter\\Documents\\Projekte\\Zähler\\tc_all_new.db', echo = False)

In [3]:
conn = engine.connect()

#### So komme ich an die Tabellen in der DB!

In [4]:
inspector = inspect(engine)

In [5]:
tab = inspector.get_table_names()
tab = tab[:-1]

#### Und so die Spaltennamen und Typen in einer Tabelle!

In [6]:
columns_table = inspector.get_columns('tcdata_CH_0084_01') 


for c in columns_table :
    print(c['name'], c['type'])

time TEXT
vfr11 REAL
s12 REAL
vfr21 REAL
s22 REAL
vfr1 REAL
s2 REAL


#### Es ist gar nicht so einfach, in SQLalchemy auf Tabellen einer bestehenden DB zuzugreifen. Bei Verwendung von des ORM kann "automate" zur Anwendung kommen. Ohne ORM geht es wie folgt:

In [7]:
metadata = db.MetaData(bind=engine)

In [8]:
table = db.Table('tcdata_CH_0084_01', metadata, autoload = True)

#### Voila, mit der so definierten Tabelle kann ich die Abfragen starten!

In [9]:
q = db.select(table)
result = conn.execute(q)
row = result.fetchmany(6)
row

[('2021-07-27 19:53:00+02:00', 900.0, 68.9000015, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 19:58:00+02:00', 840.0, 65.5, 60.0, 73.0, 0.0, 0.0),
 ('2021-07-27 20:03:00+02:00', 840.0, 67.8000031, 60.0, 72.0, 0.0, 0.0),
 ('2021-07-27 20:08:00+02:00', 600.0, 79.3000031, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 20:13:00+02:00', 720.0, 72.3000031, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 20:18:00+02:00', 780.0, 73.9000015, 0.0, 0.0, 0.0, 0.0)]

In [10]:
from sqlalchemy.sql import func, and_, desc, alias

In [15]:
q = db.select(table).where(and_(func.strftime('%m', table.c.time) == '11' , func.strftime('%d', table.c.time) == '27')).group_by(func.strftime('%H', table.c.time))
result = conn.execute(q)
row = result.fetchmany(25)
row

[('2021-11-27 01:03:00+01:00', 480.0, 72.3000031, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 02:03:00+01:00', 180.0, 77.0, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 03:03:00+01:00', 120.0, 61.0, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 04:03:00+01:00', 300.0, 64.8000031, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 05:00:00+01:00', 120.0, 67.5, 60.0, 70.0, 0.0, 0.0),
 ('2021-11-27 06:00:00+01:00', 360.0, 68.6999969, 60.0, 68.0, 0.0, 0.0),
 ('2021-11-27 07:00:00+01:00', 540.0, 73.5999985, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 08:03:00+01:00', 960.0, 68.3000031, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 09:03:00+01:00', 1320.0, 66.9000015, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 10:03:00+01:00', 1440.0, 65.5999985, 120.0, 61.5, 0.0, 0.0),
 ('2021-11-27 11:03:00+01:00', 1500.0, 64.8000031, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 12:03:00+01:00', 2400.0, 58.5, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 13:03:00+01:00', 1800.0, 65.5999985, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 14:03:00+01:00', 1620.0, 68.6999969, 0.0, 0.0, 0.0, 0.0),
 ('2021-11-27 15:

#### Abfrage über mehrer Zeilen

In [18]:
q = db.select(func.strftime('%w', table.c.time), func.max(table.c.s12)).where(and_(func.strftime('%m', table.c.time) == '11')). \
group_by(func.strftime('%w', table.c.time))
result = conn.execute(q)
row = result.fetchmany(25)
row

[('0', 105.0),
 ('1', 112.0),
 ('2', 102.0),
 ('3', 105.0),
 ('4', 108.0),
 ('5', 97.0),
 ('6', 131.0)]

#### order_by

In [19]:
q = db.select(func.strftime('%w', table.c.time), func.max(table.c.s12)).where(and_(func.strftime('%m', table.c.time) == '11')). \
group_by(func.strftime('%w', table.c.time)).order_by(table.c.s12)
result = conn.execute(q)
row = result.fetchmany(25)
row

[('5', 97.0),
 ('2', 102.0),
 ('0', 105.0),
 ('3', 105.0),
 ('4', 108.0),
 ('1', 112.0),
 ('6', 131.0)]

In [20]:
q = db.select(func.strftime('%w', table.c.time), func.max(table.c.s12)).where(and_(func.strftime('%m', table.c.time) == '11')). \
group_by(func.strftime('%w', table.c.time)).order_by(desc(table.c.s12))
result = conn.execute(q)
row = result.fetchmany(25)
row

[('6', 131.0),
 ('1', 112.0),
 ('4', 108.0),
 ('3', 105.0),
 ('0', 105.0),
 ('2', 102.0),
 ('5', 97.0)]

#### Intro eines Alias für table

In [21]:
t = table.alias("a")

In [22]:
q = db.select(t)
result = conn.execute(q)
row = result.fetchmany(5)
row

[('2021-07-27 19:53:00+02:00', 900.0, 68.9000015, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 19:58:00+02:00', 840.0, 65.5, 60.0, 73.0, 0.0, 0.0),
 ('2021-07-27 20:03:00+02:00', 840.0, 67.8000031, 60.0, 72.0, 0.0, 0.0),
 ('2021-07-27 20:08:00+02:00', 600.0, 79.3000031, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 20:13:00+02:00', 720.0, 72.3000031, 0.0, 0.0, 0.0, 0.0)]

In [23]:
q = db.select(t, func.strftime('%m', t.c.time).label('Monat'))
result = conn.execute(q)
row = result.fetchmany(6)
row

[('2021-07-27 19:53:00+02:00', 900.0, 68.9000015, 0.0, 0.0, 0.0, 0.0, '07'),
 ('2021-07-27 19:58:00+02:00', 840.0, 65.5, 60.0, 73.0, 0.0, 0.0, '07'),
 ('2021-07-27 20:03:00+02:00', 840.0, 67.8000031, 60.0, 72.0, 0.0, 0.0, '07'),
 ('2021-07-27 20:08:00+02:00', 600.0, 79.3000031, 0.0, 0.0, 0.0, 0.0, '07'),
 ('2021-07-27 20:13:00+02:00', 720.0, 72.3000031, 0.0, 0.0, 0.0, 0.0, '07'),
 ('2021-07-27 20:18:00+02:00', 780.0, 73.9000015, 0.0, 0.0, 0.0, 0.0, '07')]

In [24]:
q = db.select(t.c.vfr11, func.strftime('%m', t.c.time).label('Monat')).group_by('Monat')
result = conn.execute(q)
row = result.fetchmany(6)
row

[(540.0, '01'),
 (180.0, '02'),
 (0.0, '03'),
 (60.0, '04'),
 (240.0, '05'),
 (900.0, '07')]

In [25]:
q = db.select(t, func.strftime('%m', t.c.time).label('Monat'), func.count(t.c.time))
result = conn.execute(q)
row = result.fetchmany(6)
row

[('2021-07-27 19:53:00+02:00', 900.0, 68.9000015, 0.0, 0.0, 0.0, 0.0, '07', 170350)]

In [26]:
q = db.select(t, func.strftime('%m', t.c.time).label('Monat'), func.count(t.c.time)).group_by('Monat')
result = conn.execute(q)
row = result.fetchmany(14)
row

[('2022-01-01 01:03:00+01:00', 540.0, 60.4000015, 0.0, 0.0, 0.0, 0.0, '01', 17500),
 ('2022-02-01 01:03:00+01:00', 180.0, 45.0, 0.0, 0.0, 0.0, 0.0, '02', 15837),
 ('2022-03-01 04:53:00+01:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '03', 17660),
 ('2022-04-01 02:03:00+02:00', 60.0, 62.0, 0.0, 0.0, 0.0, 0.0, '04', 17276),
 ('2022-05-01 02:03:00+02:00', 240.0, 63.2999992, 0.0, 0.0, 0.0, 0.0, '05', 13862),
 ('2021-07-27 19:53:00+02:00', 900.0, 68.9000015, 0.0, 0.0, 0.0, 0.0, '07', 3407),
 ('2021-08-01 20:09:00+02:00', 600.0, 71.5999985, 0.0, 0.0, 0.0, 0.0, '08', 17286),
 ('2021-09-01 02:03:00+02:00', 120.0, 66.5, 0.0, 0.0, 0.0, 0.0, '09', 16613),
 ('2021-10-01 02:03:00+02:00', 120.0, 70.0, 0.0, 0.0, 0.0, 0.0, '10', 17238),
 ('2021-11-01 01:03:00+01:00', 180.0, 73.6999969, 0.0, 0.0, 0.0, 0.0, '11', 16612),
 ('2021-12-01 01:03:00+01:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '12', 17059)]

In [27]:
q = db.select(t.c.time, func.strftime('%H', t.c.time).label('Stunde'), func.strftime('%d', t.c.time).label('Tag'), func.count(t.c.time)).group_by('Stunde', 'Tag')
result = conn.execute(q)
row = result.fetchmany(14)
row

[('2021-09-01 02:03:00+02:00', '00', '01', 96),
 ('2021-08-02 02:00:00+02:00', '00', '02', 132),
 ('2021-09-03 02:03:00+02:00', '00', '03', 106),
 ('2021-08-04 02:03:00+02:00', '00', '04', 120),
 ('2021-08-05 02:03:00+02:00', '00', '05', 108),
 ('2021-08-06 02:03:00+02:00', '00', '06', 132),
 ('2021-08-07 02:03:00+02:00', '00', '07', 84),
 ('2021-08-08 02:04:00+02:00', '00', '08', 120),
 ('2021-08-09 02:03:00+02:00', '00', '09', 120),
 ('2021-08-10 02:03:00+02:00', '00', '10', 120),
 ('2021-08-11 02:03:00+02:00', '00', '11', 120),
 ('2021-08-12 02:03:00+02:00', '00', '12', 120),
 ('2021-08-13 02:03:00+02:00', '00', '13', 119),
 ('2021-08-14 02:03:00+02:00', '00', '14', 108)]

In [28]:
q = db.select(t.c.time, func.strftime('%H', t.c.time).label('Stunde'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%w', t.c.time), func.count(t.c.time)). \
group_by(func.strftime('%w', t.c.time))
result = conn.execute(q)
row = result.fetchmany(14)
row

[('2021-08-01 20:09:00+02:00', '18', '01', '0', 23336),
 ('2021-08-02 02:00:00+02:00', '00', '02', '1', 24313),
 ('2021-07-27 19:53:00+02:00', '17', '27', '2', 24001),
 ('2021-07-28 02:03:00+02:00', '00', '28', '3', 24658),
 ('2021-07-29 02:03:00+02:00', '00', '29', '4', 25082),
 ('2021-07-30 02:03:00+02:00', '00', '30', '5', 24370),
 ('2021-07-31 02:03:00+02:00', '00', '31', '6', 24590)]

#### Abfragen mit Zeiträumen als Kriterium

In [29]:
q = db.select(t.c.time, func.strftime('%H', t.c.time).label('Stunde'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%w', t.c.time), func.count(t.c.time)). \
where(t.c.time >= '2021-11-01 20:09:00+02:00'). \
group_by(func.strftime('%w', t.c.time))
result = conn.execute(q)
row = result.fetchmany(14)
row

[('2021-11-07 04:53:00+01:00', '03', '07', '0', 15566),
 ('2021-11-01 20:13:00+01:00', '19', '01', '1', 16612),
 ('2021-11-02 01:03:00+01:00', '00', '02', '2', 16966),
 ('2021-11-03 01:03:00+01:00', '00', '03', '3', 16394),
 ('2021-11-04 01:03:00+01:00', '00', '04', '4', 17014),
 ('2021-11-05 01:03:00+01:00', '00', '05', '5', 16364),
 ('2021-11-06 01:03:00+01:00', '00', '06', '6', 16374)]

In [33]:
q = db.select(t.c.time, func.strftime('%H', t.c.time).label('Stunde'), func.strftime('%d', t.c.time).label('Tag'), func.strftime('%w', t.c.time).label('Wochentag'), func.count(t.c.time)). \
where(and_(t.c.time >= '2021-11-01 20:09:00+02:00', t.c.time <= '2021-11-09 20:09:00+02:00')). \
group_by(func.strftime('%w', t.c.time))
result = conn.execute(q)
row = result.fetchmany(14)
row

[('2021-11-07 04:53:00+01:00', '03', '07', '0', 221),
 ('2021-11-01 20:13:00+01:00', '19', '01', '1', 624),
 ('2021-11-02 01:03:00+01:00', '00', '02', '2', 1090),
 ('2021-11-03 01:03:00+01:00', '00', '03', '3', 574),
 ('2021-11-04 01:03:00+01:00', '00', '04', '4', 573),
 ('2021-11-05 01:03:00+01:00', '00', '05', '5', 574),
 ('2021-11-06 01:03:00+01:00', '00', '06', '6', 334)]

In [34]:
df = pd.read_sql_query(q, conn)

In [35]:
df.head(10)

,time,Stunde,Tag,Wochentag,count_1
0,2021-11-07 04:53:00+01:00,03,07,0,221
1,2021-11-01 20:13:00+01:00,19,01,1,624
2,2021-11-02 01:03:00+01:00,00,02,2,1090
3,2021-11-03 01:03:00+01:00,00,03,3,574
4,2021-11-04 01:03:00+01:00,00,04,4,573
5,2021-11-05 01:03:00+01:00,00,05,5,574
6,2021-11-06 01:03:00+01:00,00,06,6,334
